# Generation of candidate suspect list

- Julia M. Gauglitz
- Wout Bittremieux

In [ ]:
import functools
import logging
import math
from typing import Dict

import matplotlib.pyplot as plt
import matplotlib.ticker as mticker
import numpy as np
import pandas as pd
import scipy.signal as ssignal
import seaborn as sns
import tqdm.notebook as tqdm
from sklearn.neighbors.kde import KernelDensity

In [ ]:
# Plot styling.
plt.style.use(['seaborn-white', 'seaborn-paper'])
plt.rc('font', family='serif')
sns.set_palette('Set1')
sns.set_context('paper', font_scale=1.3)

In [ ]:
logging.basicConfig(format='{asctime} [{levelname}/{processName}] {message}',
                    style='{', level=logging.INFO)
logging.captureWarnings(True)
logger = logging.getLogger('suspect_list')
logger.setLevel(logging.INFO)

### Data wrangling functionality

In [ ]:
mass_shift_annotations = pd.read_csv(
    'https://docs.google.com/spreadsheets/d/'
    '1-xh2XpSqdsa4yU-ATpDRxmpZEH6ht982jCCATFOpkyM/'
    'export?format=csv&gid=566878567')

In [ ]:
def filter_ids(ids: pd.DataFrame, max_ppm: float = 20,
               min_shared_peaks: int = 6) -> pd.DataFrame:
    """
    Filter high-quality identifications according to the given maximum ppm
    deviation and minimum number of shared peaks. Identifications without an
    InChI will be omitted as well.
    
    Arguments
    ---------
    ids : pd.DataFrame
        The tabular identifications retrieved from GNPS.
    max_ppm : float
        The maximum ppm deviation.
    min_shared_peaks : int
        The minimum number of shared peaks.
    
    Returns
    -------
    pd.DataFrame
        The identifications retained after filtering.
    """
    return (ids[(ids['MZErrorPPM'].abs() <= max_ppm) &
                (ids['SharedPeaks'] >= min_shared_peaks)]
            .dropna(subset=['INCHI']))

In [ ]:
def filter_pairs(pairs: pd.DataFrame, min_cosine: float = 0.8) \
        -> pd.DataFrame:
    """
    Only consider pairs with a cosine similarity that exceeds the given
    cosine threshold.
    
    Arguments
    ---------
    pairs : pd.DataFrame
        The tabular pairs retrieved from GNPS.
    min_cosine : float
        The minimum cosine used to retain high-quality pairs.
    
    Returns
    -------
    pd.DataFrame
        The pairs filtered by minimum cosine similarity.
    """
    return pairs[pairs['Cosine'] >= min_cosine]

In [ ]:
def filter_summary(summary: pd.DataFrame) -> pd.DataFrame:
    """
    For each cluster select as representative the scan with the highest
    precursor intensity.
    
    Arguments
    ---------
    summary : pd.DataFrame
        The tabular summary retrieved from GNPS.
    
    Returns
    -------
    pd.DataFrame
        The summary with only the scans with the highest precursor intensity
        for each cluster.
    """
    summary = (summary.reindex(summary.groupby(['dataset', 'cluster index'])
                               ['sum(precursor intensity)'].idxmax())
               .dropna().reset_index(drop=True)
               [['dataset', 'cluster index', 'parent mass', 'ScanNumber',
                 'Original_Path']])
    summary['cluster index'] = summary['cluster index'].astype(int)
    summary['ScanNumber'] = summary['ScanNumber'].astype(int)
    return summary

In [ ]:
def generate_suspects(ids: pd.DataFrame, pairs: pd.DataFrame,
                      summary: pd.DataFrame) -> pd.DataFrame:
    """
    Generate suspects from identifications and aligned spectra pairs.
    Provenance about the spectra pairs is added from the summary.
    
    Arguments
    ---------
    ids : pd.DataFrame
        The filtered identifications.
    pairs : pd.DataFrame
        The filtered pairs.
    summary : pd.DataFrame
        The filtered summary information for the clusters.
    
    Returns
    -------
    pd.DataFrame
        A DataFrame with information about both spectra forming a suspect
        identification.
    """
    # Form suspects of library and unidentified spectra pairs.
    suspects = pd.concat([
        pd.merge(pairs, ids, left_on=['dataset', 'CLUSTERID1'],
                 right_on=['dataset', '#Scan#'])
        .drop(columns=['CLUSTERID1'])
        .rename(columns={'CLUSTERID2': 'SuspectIndex'}),
        pd.merge(pairs, ids, left_on=['dataset', 'CLUSTERID2'],
                 right_on=['dataset', '#Scan#'])
        .drop(columns=['CLUSTERID2'])
        .rename(columns={'CLUSTERID1': 'SuspectIndex'})],
        ignore_index=True, sort=False).dropna(axis=1)
    
    # TODO: Properly handle this warning.
    if not suspects['SuspectIndex'].is_unique:
        logger.warning('Multiple analog matches per suspect scan found')
    
    # Add provenance information for the library and suspect scans.
    suspects = (suspects[['dataset', 'INCHI', 'Compound_Name', 'Adduct',
                          'Cosine', 'Precursor_MZ', 'SpectrumID', '#Scan#',
                          'SuspectIndex']]
                .rename(columns={'Compound_Name': 'CompoundName',
                                 'Precursor_MZ': 'LibraryPrecursorMZ',
                                 'SpectrumID': 'LibraryID',
                                 '#Scan#': 'ClusterScanNr'}))
    suspects = (pd.merge(suspects, summary,
                         left_on=['dataset', 'SuspectIndex'],
                         right_on=['dataset', 'cluster index'])
                .drop(columns=['SuspectIndex', 'cluster index'])
                .rename(columns={'parent mass': 'SuspectPrecursorMZ',
                                 'Original_Path': 'SuspectPath',
                                 'ScanNumber': 'SuspectScanNr'}))
    return suspects.drop(columns=['dataset'])

In [ ]:
def assign_filter_putative_mass_shifts(
        suspects: pd.DataFrame, mass_shift_annotations: pd.DataFrame,
        min_delta_mz: float = 1, interval_width: float = 1.0,
        bin_width: float = 0.02, prominence: float = 0.5) -> pd.DataFrame:
    suspects['DeltaMZ'] = \
        suspects['LibraryPrecursorMZ'] - suspects['SuspectPrecursorMZ']
    # Remove suspects without a mass shift.
    suspects = suspects[suspects['DeltaMZ'].abs() > min_delta_mz].copy()
    # Assign putative identifications to the mass shifts.
    kde = KernelDensity(kernel='gaussian', bandwidth=bin_width)
    for mz in np.arange(math.floor(suspects['DeltaMZ'].min()),
                        math.ceil(suspects['DeltaMZ'].max())):
        mask = suspects.index[suspects['DeltaMZ'].between(
            mz - interval_width / 2, mz + interval_width / 2)]
        if len(mask) == 0:
            continue
        bins = (np.linspace(mz - interval_width / 2,
                            mz + interval_width / 2,
                            int(interval_width / bin_width) + 1)
                + bin_width / 2)
        kde.fit(suspects.loc[mask, 'DeltaMZ'].values.reshape(-1, 1))
        density = np.exp(kde.score_samples(bins.reshape(-1, 1)))
        peaks_i, prominences = ssignal.find_peaks(density,
                                                  prominence=prominence)
        order = np.argsort(prominences['prominences'])[::-1]
        for delta_mz, start_i, stop_i in zip(
                bins[peaks_i[order]],
                prominences['left_bases'][order],
                prominences['right_bases'][order]):
            putative_id = mass_shift_annotations[
                (mass_shift_annotations['mz delta'].abs()
                 - abs(delta_mz)).abs() < bin_width]
            mask_putative_id = suspects.loc[mask].index[
                suspects.loc[mask, 'DeltaMZ'].between(
                    bins[start_i], bins[stop_i])]
            suspects.loc[mask_putative_id, 'GroupDeltaMZ'] = delta_mz
            suspects.loc[mask_putative_id, 'AtomicDifference'] = \
                '|'.join(putative_id['atomic difference'])
            suspects.loc[mask_putative_id, 'Rationale'] = \
                '|'.join(putative_id['rationale'].fillna('unknown'))
    
    # Only use the top suspect (by cosine score) per combination of library
    # spectra and putative identification.
    suspects = (suspects.sort_values(['Cosine'], ascending=False)
                .drop_duplicates(['CompoundName', 'GroupDeltaMZ']))
    
    return (suspects.sort_values(['CompoundName', 'GroupDeltaMZ'])
            .reset_index(drop=True)
            [['INCHI', 'CompoundName', 'Adduct', 'DeltaMZ', 'GroupDeltaMZ',
              'AtomicDifference', 'Rationale', 'Cosine',
              'LibraryPrecursorMZ', 'LibraryID', 'ClusterScanNr',
              'SuspectPrecursorMZ', 'SuspectScanNr', 'SuspectPath']])

## Generate suspect library matches

Criteria to form a suspect:

- Identification ≤ 20 ppm.
- Identification ≥ 6 shared peaks.
- Identification has to include InChI.
- Cosine ≥ 0.8.
- The spectrum with maximal intensity is chosen as cluster representative.

Criteria for putative suspect explanations:

- Corrected delta _m_/_z_'s for common shifts are determined using kernel density estimation with bandwith 0.02 within each 1 _m_/_z_ window (centered around unit _m_/_z_'s).
- The corrected delta _m_/_z_ is compared to known mass shifts with a 0.02 _m_/_z_ threshold.

In [ ]:
ftp_prefix = 'ftp://massive.ucsd.edu/MSV000084314/other'
ids, pairs, summary = [], [], []
logger.info('Retrieve cluster information')
msv_ids = pd.read_csv('datasets.csv', header=None).squeeze()
msv_ids_retry = []
while len(msv_ids) > 0:
    for msv_id in tqdm.tqdm(msv_ids, desc='Datasets read', unit='dataset'):
        try:
            ids.append(pd.read_csv(
                f'{ftp_prefix}/IDENTIFICATIONS/{msv_id}_identifications.tsv',
                sep='\t', usecols=[
                    'Compound_Name', 'Adduct', 'Precursor_MZ', 'INCHI',
                    'SpectrumID', 'LibraryQualityString', '#Scan#',
                    'MZErrorPPM', 'SharedPeaks']))
            ids[-1]['dataset'] = msv_id
            pairs.append(pd.read_csv(
                f'{ftp_prefix}/PAIRS/{msv_id}_pairs.tsv', sep='\t',
                usecols=['CLUSTERID1', 'CLUSTERID2', 'Cosine']))
            pairs[-1]['dataset'] = msv_id
            summary.append(pd.read_csv(
                f'{ftp_prefix}/CLUSTERSUMMARY/{msv_id}_summary.tsv',
                sep='\t', usecols=[
                    'cluster index', 'sum(precursor intensity)',
                    'parent mass', 'Original_Path', 'ScanNumber']))
            summary[-1]['dataset'] = msv_id
        except ValueError:
            logger.warning("Couldn't process dataset %s", msv_id)
        except IOError:
            msv_ids_retry.append(msv_id)
    msv_ids = msv_ids_retry
    msv_ids_retry = []
    
logger.info('Compile suspect pairs')
ids = filter_ids(pd.concat(ids, ignore_index=True))
pairs = filter_pairs(pd.concat(pairs, ignore_index=True))
summary = filter_summary(pd.concat(summary, ignore_index=True))
suspects = generate_suspects(ids, pairs, summary)
logger.info('Assign putative explanations to mass shifts')
suspects = assign_filter_putative_mass_shifts(suspects,
                                              mass_shift_annotations)

### Explore delta masses

In [ ]:
delta_mzs = (suspects['GroupDeltaMZ'].value_counts().reset_index()
             .rename(columns={'index': 'DeltaMZ', 'GroupDeltaMZ': 'count'})
            .sort_values('count', ascending=False))

In [ ]:
width = 7
height = width / 1.618
fig, ax = plt.subplots(figsize=(width * 1.5, height / 1.5))

ax.bar(delta_mzs['DeltaMZ'], delta_mzs['count'], width=0.4, color='black')

ax.set_xlim(-300, 300)

sns.despine(ax=ax)

ax.set_xlabel('Delta m/z')
ax.set_ylabel(f'Number of suspects')

plt.savefig('delta_mz.png', dpi=300, bbox_inches='tight')
plt.show()
plt.close()

In [ ]:
delta_mz_count_cdf = np.cumsum(delta_mzs['count']) / delta_mzs['count'].sum()
delta_mz_count_cdf = np.insert(delta_mz_count_cdf.values, 0, 0)

prop_threshold = 0.8
rank_threshold = np.argmax(delta_mz_count_cdf > prop_threshold)
count_threshold = delta_mzs['count'].iloc[rank_threshold]

In [ ]:
width = 7
height = width / 1.618
fig, ax = plt.subplots(figsize=(width, height))

ax.plot(np.arange(len(delta_mz_count_cdf)), delta_mz_count_cdf)

ax.plot([rank_threshold, rank_threshold], [0, prop_threshold], 'k--')
ax.plot([0, rank_threshold], [prop_threshold, prop_threshold], 'k--')

ax.yaxis.set_major_formatter(mticker.PercentFormatter(1))

ax.set_xlim(0, ax.get_xlim()[1])
ax.set_ylim(0, ax.get_ylim()[1])

sns.despine(ax=ax)

ax.set_xlabel('Delta m/z frequency rank')
ax.set_ylabel('Proportion of suspects')

plt.savefig('delta_mz_count_cdf.png', dpi=300, bbox_inches='tight')
plt.show()
plt.close()

In [ ]:
print(f'Frequency threshold at {prop_threshold:.0%} CDF = {count_threshold}')

### Export frequent/infrequent suspect libraries

In [ ]:
suspects_freq = suspects[suspects['GroupDeltaMZ'].isin(
    delta_mzs[delta_mzs['count'] >= count_threshold]['DeltaMZ'])]
suspects_infreq = suspects[~suspects['GroupDeltaMZ'].isin(
    delta_mzs[delta_mzs['count'] >= count_threshold]['DeltaMZ'])]

In [ ]:
print(f'Number of frequent suspects: {len(suspects_freq)} '
      f'({suspects_freq["GroupDeltaMZ"].nunique()} unique m/z deltas)')
print(f'Number of infrequent suspects: {len(suspects_infreq)} '
      f'({suspects_infreq["GroupDeltaMZ"].nunique()} unique m/z deltas)')

In [ ]:
suspects_freq.to_csv('suspect_library_freq.csv', index=False)
suspects_infreq.to_csv('suspect_library_infreq.csv', index=False)